In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Path to the file
file_path = '/data/chats/igk4wd/workspace/uploads/Teste Power BI - Deveras RH.xlsx'

# List all sheets in the Excel file
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
print("Excel sheets:", sheet_names)

# Create a dictionary to store dataframes from each sheet
dfs = {}

# Read each sheet and store in the dictionary
for sheet in sheet_names:
    dfs[sheet] = pd.read_excel(file_path, sheet_name=sheet)
    print(f"\n--- Sheet: {sheet} ---")
    print(f"Shape: {dfs[sheet].shape}")
    print("Column names:", list(dfs[sheet].columns))
    print("Data types:\n", dfs[sheet].dtypes)
    
    # Display first few rows
    print("First 3 rows:\n", dfs[sheet].head(3))
    
    # Check for missing values
    missing_values = dfs[sheet].isnull().sum()
    if missing_values.sum() > 0:
        print("Missing values:\n", missing_values[missing_values > 0])
    else:
        print("No missing values found.")
    
    # Check for duplicates
    duplicates = dfs[sheet].duplicated().sum()
    print(f"Number of duplicate rows: {duplicates}")

Excel sheets: ['BASE DE DADOS', 'Orientações', 'Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']

--- Sheet: BASE DE DADOS ---
Shape: (16, 2)
Column names: ['CÓDIGO DO CLIENTE:', 'Unnamed: 1']
Data types:
 CÓDIGO DO CLIENTE:     object
Unnamed: 1            float64
dtype: object
First 3 rows:
           CÓDIGO DO CLIENTE:  Unnamed: 1
0  NOME COMPLETO DO CLIENTE:         NaN
1                       CPF:         NaN
2                  ENDEREÇO:         NaN
Missing values:
 Unnamed: 1    12
dtype: int64
Number of duplicate rows: 0

--- Sheet: Orientações ---
Shape: (17, 2)
Column names: ['Unnamed: 0', 'Unnamed: 1']
Data types:
 Unnamed: 0    object
Unnamed: 1    object
dtype: object
First 3 rows:
   Unnamed: 0 Unnamed: 1
0        NaN        NaN
1        NaN        NaN
2        NaN        NaN
Missing values:
 Unnamed: 0     9
Unnamed: 1    11
dtype: int64
Number of duplicate rows: 8

--- Sheet: Alphaville ---
Shape: (1651, 45)
Column names: ['Nº CONTRATO (SISTEMA GERA)', 'STATUS', 'CÓD CL

In [2]:
# Now let's continue our analysis by focusing on the remaining sheets
# and convert the VALOR column to numeric for analysis

# Let's look at the remaining sheets in more depth
location_sheets = ['Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']

# Dictionary to store cleaned data frames
cleaned_dfs = {}

for sheet in location_sheets:
    if sheet in dfs:
        print(f"\n--- Detailed analysis of {sheet} sheet ---")
        
        # Convert VALOR column to numeric (removing any currency symbols, etc.)
        if 'VALOR' in dfs[sheet].columns:
            # First, check the current format
            print(f"VALOR column sample values: {dfs[sheet]['VALOR'].head(3).tolist()}")
            
            # Try to convert to numeric, coercing errors to NaN
            dfs[sheet]['VALOR_NUMERIC'] = pd.to_numeric(
                dfs[sheet]['VALOR'].str.replace('R\$', '').str.replace('.', '').str.replace(',', '.'),
                errors='coerce'
            )
            
            # Check if the conversion worked
            print(f"Converted VALOR sample: {dfs[sheet]['VALOR_NUMERIC'].head(3).tolist()}")
            print(f"NaN values after conversion: {dfs[sheet]['VALOR_NUMERIC'].isna().sum()}")
        
        # Check for data consistency between sheets
        print(f"Column count: {dfs[sheet].shape[1]}")
        
        # Analyze key statistics
        numeric_columns = dfs[sheet].select_dtypes(include=[np.number])
        if not numeric_columns.empty:
            print("\nNumeric columns statistics:")
            print(numeric_columns.describe().T)
        
        # Analyze categorical columns
        categorical_columns = dfs[sheet].select_dtypes(include=['object'])
        if not categorical_columns.empty:
            print("\nCategorical columns unique values count:")
            for col in categorical_columns.columns[:5]:  # Limiting to first 5 columns to avoid too much output
                print(f"{col}: {categorical_columns[col].nunique()} unique values")
                print(f"Top 3 values: {categorical_columns[col].value_counts().nlargest(3)}")
        
        # Check date columns for temporal range
        date_columns = dfs[sheet].select_dtypes(include=['datetime64'])
        if not date_columns.empty:
            print("\nDate ranges:")
            for col in date_columns.columns:
                min_date = date_columns[col].min()
                max_date = date_columns[col].max()
                print(f"{col}: {min_date} to {max_date}")
        
        # Store the dataframe with converted values
        cleaned_dfs[sheet] = dfs[sheet]

# Check for inconsistencies between location sheets
print("\n--- Checking inconsistencies between sheets ---")
if len(location_sheets) > 1 and all(sheet in dfs for sheet in location_sheets):
    # Check if all sheets have the same columns
    all_columns = [set(dfs[sheet].columns) for sheet in location_sheets]
    common_columns = set.intersection(*all_columns)
    
    print(f"Common columns across all location sheets: {len(common_columns)}")
    
    # Check for differences in columns
    for sheet in location_sheets:
        diff_columns = set(dfs[sheet].columns) - common_columns
        if diff_columns:
            print(f"{sheet} has {len(diff_columns)} unique columns: {diff_columns}")


--- Detailed analysis of Alphaville sheet ---
VALOR column sample values: [41545.78, 54270.99, 43074.45]
Converted VALOR sample: [nan, nan, nan]
NaN values after conversion: 1651
Column count: 46

Numeric columns statistics:
                             count          mean           std      min  \
Nº CONTRATO (SISTEMA GERA)  1651.0    826.000000    476.746963      1.0   
CÓD CLIENTE                 1651.0   8260.000000   4767.469629     10.0   
 NF                         1646.0   4019.770960    596.754145   2965.0   
QUINZENA                    1647.0      1.485124      0.499930      1.0   
Nº SESSÕES                  1651.0     14.933374      3.530188      9.0   
VALOR FINANCIADO            1651.0  47518.230999  21057.175113   3000.0   
VALOR PAGAMENTO FACTORING      9.0  37654.580000  14857.632211  10000.0   
VALOR_NUMERIC                  0.0           NaN           NaN      NaN   

                                 25%       50%        75%        max  
Nº CONTRATO (SISTEMA GERA) 

In [3]:
# Fix the VALOR conversion issue and complete the analysis of all sheets
import re

# Function to properly convert VALOR to numeric
def convert_to_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return value
    
    # Remove non-numeric characters except for decimal separator
    # First, standardize the value format
    value_str = str(value)
    # Remove currency symbols, spaces, etc.
    cleaned = re.sub(r'[^\d,.]', '', value_str)
    # Convert Brazilian/European format (1.234,56) to US format (1234.56)
    if ',' in cleaned and '.' in cleaned:
        cleaned = cleaned.replace('.', '').replace(',', '.')
    elif ',' in cleaned:
        cleaned = cleaned.replace(',', '.')
    
    try:
        return float(cleaned)
    except:
        return np.nan

# Create a comprehensive comparison of all location sheets
location_sheets = ['Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']
summary_data = {}

# Process each sheet
for sheet in location_sheets:
    if sheet in dfs:
        df = dfs[sheet].copy()
        
        print(f"\n--- Processing {sheet} sheet ---")
        
        # Convert VALOR to numeric
        if 'VALOR' in df.columns:
            # Apply the conversion function
            df['VALOR_NUMERIC'] = df['VALOR'].apply(convert_to_numeric)
            print(f"Converted VALOR sample: {df['VALOR_NUMERIC'].head(3).tolist()}")
            print(f"NaN values after conversion: {df['VALOR_NUMERIC'].isna().sum()}")
            
            # Basic statistics on VALOR
            if df['VALOR_NUMERIC'].notna().any():
                print(f"VALOR statistics: Min={df['VALOR_NUMERIC'].min()}, Max={df['VALOR_NUMERIC'].max()}, Mean={df['VALOR_NUMERIC'].mean()}")
        
        # Check for inconsistent column names (case, spacing, etc.)
        column_names = df.columns.tolist()
        cleaned_column_names = [col.strip().upper() for col in column_names]
        if len(set(cleaned_column_names)) != len(column_names):
            print("Warning: Potential inconsistent column naming (case or spacing)")
            duplicates = [col for col in cleaned_column_names if cleaned_column_names.count(col) > 1]
            if duplicates:
                print(f"Duplicate columns after normalization: {set(duplicates)}")
        
        # Check for inconsistencies in categorical data
        if 'UNIDADE' in df.columns:
            print(f"UNIDADE values in {sheet}: {df['UNIDADE'].unique()}")
        
        # Store sheet-specific data for summary
        summary_data[sheet] = {
            'row_count': df.shape[0],
            'column_count': df.shape[1],
            'missing_values': df.isnull().sum().sum(),
            'date_range': None
        }
        
        # Get date range if available
        date_cols = df.select_dtypes(include=['datetime64']).columns
        if len(date_cols) > 0:
            min_date = df[date_cols].min().min()
            max_date = df[date_cols].max().max()
            summary_data[sheet]['date_range'] = (min_date, max_date)
        
        # Store the cleaned dataframe
        cleaned_dfs[sheet] = df

# Summarize the findings
print("\n--- Summary of Data Analysis ---")
print(f"Total sheets analyzed: {len(location_sheets)}")
for sheet, data in summary_data.items():
    print(f"\n{sheet}:")
    print(f"  - Records: {data['row_count']}")
    print(f"  - Fields: {data['column_count']}")
    print(f"  - Missing values: {data['missing_values']}")
    if data['date_range']:
        print(f"  - Date range: {data['date_range'][0]} to {data['date_range'][1]}")

# Identify structure inconsistencies across sheets
print("\n--- Structural Inconsistencies ---")
all_columns = [set(dfs[sheet].columns) for sheet in location_sheets if sheet in dfs]
if all_columns:
    common_columns = set.intersection(*all_columns)
    all_columns_flat = set().union(*all_columns)
    
    print(f"Total unique columns across all sheets: {len(all_columns_flat)}")
    print(f"Common columns across all sheets: {len(common_columns)}")
    
    # Find columns unique to specific sheets
    for i, sheet in enumerate(location_sheets):
        if sheet in dfs:
            unique_cols = set(dfs[sheet].columns) - common_columns
            if unique_cols:
                print(f"{sheet} has {len(unique_cols)} unique columns: {unique_cols}")
    
    # Check type inconsistencies in common columns
    type_inconsistencies = []
    for col in common_columns:
        col_types = {}
        for sheet in location_sheets:
            if sheet in dfs and col in dfs[sheet].columns:
                col_type = str(dfs[sheet][col].dtype)
                if col_type not in col_types:
                    col_types[col_type] = []
                col_types[col_type].append(sheet)
        
        if len(col_types) > 1:
            type_inconsistencies.append((col, col_types))
    
    if type_inconsistencies:
        print("\nType inconsistencies in common columns:")
        for col, types in type_inconsistencies:
            print(f"Column '{col}' has different types:")
            for type_name, sheets in types.items():
                print(f"  - {type_name}: in sheets {', '.join(sheets)}")


--- Processing Alphaville sheet ---
Converted VALOR sample: [41545.78, 54270.99, 43074.45]
NaN values after conversion: 21
VALOR statistics: Min=3000.0, Max=124526.12, Mean=47585.67782822086
UNIDADE values in Alphaville: ['ALPHAVILLE' 'alpha ']

--- Processing BH sheet ---
Converted VALOR sample: [51231.24, 47328.33, 57888.45]
NaN values after conversion: 52
VALOR statistics: Min=8.0, Max=84354.34, Mean=28020.540530971128
UNIDADE values in BH: ['BELO HORIZONTE' 'OSASCO' 'ALPHAVILLE' 'BSB']

--- Processing SJC sheet ---
Converted VALOR sample: [20140120092136.0, 20160529185736.0, 20160829092136.0]
NaN values after conversion: 18
VALOR statistics: Min=2600.0, Max=26030602074048.0, Mean=11384180980684.389
UNIDADE values in SJC: ['DOMICILIAR' 'SJC' 'MORUMBI' 'LINS']

--- Processing PAULINIA sheet ---
Converted VALOR sample: [78661.77, 55947.52, 54432.92]
NaN values after conversion: 10
VALOR statistics: Min=6167.61, Max=21600212072624.0, Mean=1943374697564.2173
UNIDADE values in PAULINIA:

In [4]:
# Standardize and clean the datasets across all sheets
# 1. Create a mapping for column name standardization
# 2. Standardize data types
# 3. Consolidate all data into a single clean dataset

# Define standard column names mapping
# This will standardize names across sheets and fix inconsistencies
column_mapping = {
    # Contract and client info
    'Nº CONTRATO (SISTEMA GERA)': 'CONTRATO_ID',
    'Nº CONTRATO (SISTEMA GERA)2': 'CONTRATO_ID',
    'CÓD CLIENTE': 'CLIENTE_ID',
    'PACIENTYES': 'PACIENTE_NOME',
    'PACIENTE 2': 'PACIENTE_NOME',
    'CLIENTES': 'PACIENTE_NOME',
    'DATA DO CONTRATO': 'DATA_CONTRATO',
    
    # Location and operational info
    'UNIDADE': 'UNIDADE',
    'OPERADORA': 'OPERADORA',
    'STATUS': 'STATUS',
    
    # Financial data
    ' NF': 'NF',
    'VALOR': 'VALOR_BRUTO',
    'VALOR_NUMERIC': 'VALOR',
    'VALOR FINANCIADO': 'VALOR_FINANCIADO',
    'VALOR PAGAMENTO CLÍNICA': 'VALOR_PAGAMENTO_CLINICA',
    'VALOR PAGO': 'VALOR_PAGO',
    'VALOR GLOSADO': 'VALOR_GLOSADO',
    'VALOR PAGAMENTO FACTORING': 'VALOR_PAGAMENTO_FACTORING',
    'CONTAS A RECEBER': 'CONTAS_A_RECEBER',
    
    # Date information
    'DATA EMISSÃO NF': 'DATA_EMISSAO_NF',
    'COMPETÊNCIA': 'COMPETENCIA',
    'DATA FINANCIAMENTO': 'DATA_FINANCIAMENTO',
    'DATA PAGAMENTO CLÍNICA': 'DATA_PAGAMENTO_CLINICA',
    'DATA PROTOCOLO': 'DATA_PROTOCOLO',
    'DATA PROTOCOLO ': 'DATA_PROTOCOLO',
    'DATA DE VENCIMENTO': 'DATA_VENCIMENTO',
    'DATA DO PAGAMENTO REEMBOLSO': 'DATA_PAGAMENTO_REEMBOLSO',
    'DATA DO PAGAMENTO FACTORING': 'DATA_PAGAMENTO_FACTORING',
    'DATA DA LIGAÇÃO': 'DATA_LIGACAO',
    'DATA DA RECLAMAÇÃO': 'DATA_RECLAMACAO',
    'DATA DA NIP': 'DATA_NIP',
    
    # Sessions and service information
    'Nº SESSÕES': 'NUM_SESSOES',
    'QUINZENA': 'QUINZENA',
    
    # Status fields
    'STATUS PRÉVIO DA CONTA': 'STATUS_PREVIO_CONTA',
    'STATUS ANÁLISE DE CRÉDITO': 'STATUS_ANALISE_CREDITO',
    'STATUS JURÍDICO CLIENTE': 'STATUS_JURIDICO_CLIENTE',
    'STATUS CONTAS A RECEBER': 'STATUS_CONTAS_RECEBER',
    'STATUS FINAL DA CONTA': 'STATUS_FINAL_CONTA',
    
    # Other fields
    'SOLICITANTE': 'SOLICITANTE',
    'CONTRATO ASSINADO': 'CONTRATO_ASSINADO',
    'FINANCIADO': 'FINANCIADO',
    'PAGAMENTO CLINICA': 'PAGAMENTO_CLINICA',
    'TRANSMISSÃO DA CONTA': 'TRANSMISSAO_CONTA',
    'LIGAÇÃO': 'LIGACAO',
    'RECLAMAÇÃO CHAT': 'RECLAMACAO_CHAT',
    'NIP': 'NIP',
    'ETAPAS DA JUDICILIAZAÇÃO': 'ETAPAS_JUDICIALIZACAO',
    'OBSERVAÇÕES': 'OBSERVACOES',
    '1ª APROVAÇÃO CLÍNICA GARANTIDORA': 'APROVACAO_CLINICA_GARANTIDORA',
    '2ª APROVAÇÃO FACTORING': 'APROVACAO_FACTORING',
    'USUÁRIO': 'USUARIO'
}

# Function to standardize column names
def standardize_column_names(df):
    # Rename columns based on mapping
    df_cols = df.columns.tolist()
    rename_dict = {}
    
    for col in df_cols:
        if col in column_mapping:
            rename_dict[col] = column_mapping[col]
    
    return df.rename(columns=rename_dict)

# Function to standardize data types
def standardize_data_types(df):
    # Date columns to convert - some may not exist in all sheets
    date_columns = [
        'DATA_EMISSAO_NF', 'COMPETENCIA', 'DATA_FINANCIAMENTO', 
        'DATA_PAGAMENTO_CLINICA', 'DATA_PROTOCOLO', 'DATA_VENCIMENTO',
        'DATA_PAGAMENTO_REEMBOLSO', 'DATA_PAGAMENTO_FACTORING',
        'DATA_LIGACAO', 'DATA_RECLAMACAO', 'DATA_NIP', 'DATA_CONTRATO'
    ]
    
    # Numeric columns to convert
    numeric_columns = [
        'VALOR', 'VALOR_FINANCIADO', 'VALOR_PAGAMENTO_CLINICA', 
        'VALOR_PAGO', 'VALOR_GLOSADO', 'VALOR_PAGAMENTO_FACTORING', 
        'CONTAS_A_RECEBER', 'NF', 'NUM_SESSOES', 'QUINZENA'
    ]
    
    # Convert date columns to datetime
    for col in date_columns:
        if col in df.columns:
            try:
                # Handle different date formats
                df[col] = pd.to_datetime(df[col], errors='coerce')
                
                # Handle very old dates (likely errors)
                # Replace dates before 2000 with NaT
                if df[col].dt.year.min() < 2000:
                    df.loc[df[col].dt.year < 2000, col] = pd.NaT
            except:
                print(f"Could not convert {col} to datetime in current dataframe")
    
    # Convert numeric columns to float
    for col in numeric_columns:
        if col in df.columns:
            # If the column already has numeric values, keep them
            if pd.api.types.is_numeric_dtype(df[col]):
                continue
                
            try:
                # If not already numeric, try to convert to numeric
                if col == 'VALOR' and 'VALOR_BRUTO' in df.columns:
                    # Use the already converted VALOR_NUMERIC values
                    pass
                else:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                print(f"Could not convert {col} to numeric in current dataframe")
    
    # Convert ID columns to string to ensure consistent handling
    id_columns = ['CONTRATO_ID', 'CLIENTE_ID', 'NF']
    for col in id_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    return df

# Function to standardize categorical values
def standardize_categorical_values(df):
    # Standardize UNIDADE values
    if 'UNIDADE' in df.columns:
        # Standardize case and remove extra spaces
        df['UNIDADE'] = df['UNIDADE'].str.strip().str.upper()
        
        # Map variations to standard names
        unit_mapping = {
            'ALPHA': 'ALPHAVILLE',
            'alpha': 'ALPHAVILLE',
            'BELO HORIZONTE': 'BH',
            'BSB': 'BRASILIA',
            'SJC': 'SAO JOSE DOS CAMPOS'
        }
        
        df['UNIDADE'] = df['UNIDADE'].replace(unit_mapping)
    
    # Standardize STATUS values
    if 'STATUS' in df.columns:
        df['STATUS'] = df['STATUS'].str.strip().str.upper()
    
    return df

# Function to add source sheet information
def add_sheet_info(df, sheet_name):
    df['FONTE_PLANILHA'] = sheet_name
    return df

# Process all location sheets and combine into a single dataframe
location_sheets = ['Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']
all_data = []

for sheet in location_sheets:
    if sheet in dfs:
        print(f"Standardizing {sheet} sheet...")
        
        # Make a copy to avoid modifying original data
        df = dfs[sheet].copy()
        
        # Apply standardization steps
        df = standardize_column_names(df)
        df = standardize_data_types(df)
        df = standardize_categorical_values(df)
        df = add_sheet_info(df, sheet)
        
        # Append to list for later concatenation
        all_data.append(df)
        
        print(f"  Processed {df.shape[0]} rows with {df.shape[1]} columns")

# Combine all data into a single dataframe
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Final cleanup
    # 1. Remove duplicate rows
    initial_rows = combined_df.shape[0]
    combined_df = combined_df.drop_duplicates()
    duplicate_rows = initial_rows - combined_df.shape[0]
    
    # 2. Check if client IDs are consistent
    if 'CLIENTE_ID' in combined_df.columns and 'PACIENTE_NOME' in combined_df.columns:
        client_id_mapping = combined_df.groupby('CLIENTE_ID')['PACIENTE_NOME'].nunique()
        inconsistent_ids = client_id_mapping[client_id_mapping > 1]
        
        if not inconsistent_ids.empty:
            print("\nInconsistent client IDs (multiple names for the same ID):")
            for client_id in inconsistent_ids.index:
                print(f"  Client ID {client_id} has {inconsistent_ids[client_id]} different names:")
                names = combined_df[combined_df['CLIENTE_ID'] == client_id]['PACIENTE_NOME'].unique()
                for name in names:
                    print(f"    - {name}")
    
    # Summary of standardized data
    print("\n=== STANDARDIZED DATA SUMMARY ===")
    print(f"Total records: {combined_df.shape[0]}")
    print(f"Duplicate rows removed: {duplicate_rows}")
    print(f"Final columns: {combined_df.shape[1]}")
    print(f"Column names: {combined_df.columns.tolist()}")
    
    # Check for remaining missing values
    missing_values = combined_df.isnull().sum()
    significant_missing = missing_values[missing_values > combined_df.shape[0] * 0.2]  # More than 20% missing
    if not significant_missing.empty:
        print("\nColumns with significant missing values (>20%):")
        for col, count in significant_missing.items():
            print(f"  {col}: {count} missing values ({count/combined_df.shape[0]:.1%})")
    
    # Save the standardized data to CSV
    output_path = '/data/chats/igk4wd/workspace/processed_data.csv'
    combined_df.to_csv(output_path, index=False)
    print(f"\nStandardized data saved to {output_path}")
    
    # Basic portfolio analysis
    print("\n=== PORTFOLIO ANALYSIS ===")
    if 'UNIDADE' in combined_df.columns:
        unit_counts = combined_df['UNIDADE'].value_counts()
        print("Records by operating unit:")
        for unit, count in unit_counts.items():
            print(f"  {unit}: {count} records")
    
    if 'VALOR' in combined_df.columns:
        # Filter out extreme values for analysis
        value_df = combined_df[combined_df['VALOR'] < combined_df['VALOR'].quantile(0.99)]
        
        print("\nFinancial summary by unit:")
        for unit in combined_df['UNIDADE'].unique():
            unit_data = value_df[value_df['UNIDADE'] == unit]
            if not unit_data.empty and 'VALOR' in unit_data.columns:
                total = unit_data['VALOR'].sum()
                mean = unit_data['VALOR'].mean()
                count = unit_data['VALOR'].count()
                print(f"  {unit}: {count} transactions, Total: R$ {total:,.2f}, Average: R$ {mean:,.2f}")
else:
    print("No data to standardize!")

Standardizing Alphaville sheet...
  Processed 1651 rows with 47 columns
Standardizing BH sheet...
  Processed 814 rows with 47 columns
Standardizing SJC sheet...
  Processed 2587 rows with 47 columns
Standardizing PAULINIA sheet...
  Processed 251 rows with 47 columns
Standardizing CAMPINAS sheet...
  Processed 194 rows with 46 columns

Inconsistent client IDs (multiple names for the same ID):
  Client ID 10 has 2 different names:
    - PEDRO SILVA
    - Miguel Oliveira Costa
  Client ID 100 has 3 different names:
    - PEDRO SILVA
    - JOANA MARIA-1ª HM
    - RUBENS FERREIRA-matmed
  Client ID 1000 has 4 different names:
    - Miguel costa
    - Roberto Amorin-único
    - Roberto Amorin-1ª HM
    - Felipe Cardoso Nunes
  Client ID 1001 has 2 different names:
    - Roberto Amorin-1ª matmed
    - Julio Freitas-1ª
  Client ID 1004 has 2 different names:
    - Roberto Amorin-1ª HM
    - Julio Freitas-2º
  Client ID 1008 has 2 different names:
    - Roberto Amorin-0
    - Julio Freitas-1ª

In [5]:
# Further analyze the portfolio performance by operating unit
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
from datetime import datetime

# Load the processed data
processed_data_path = '/data/chats/igk4wd/workspace/processed_data.csv'
df = pd.read_csv(processed_data_path)

# Convert date columns to datetime
date_columns = [col for col in df.columns if col.startswith('DATA_')]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Add month and year columns for time-based analysis
if 'DATA_EMISSAO_NF' in df.columns:
    df['MES'] = df['DATA_EMISSAO_NF'].dt.month
    df['ANO'] = df['DATA_EMISSAO_NF'].dt.year
    df['MES_ANO'] = df['DATA_EMISSAO_NF'].dt.strftime('%m-%Y')

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 12})

# Create a folder for visualizations
import os
viz_path = '/data/chats/igk4wd/workspace/vizualizacoes'
os.makedirs(viz_path, exist_ok=True)

# Filter out extreme values for visualization
df_viz = df.copy()
# Filter out extreme values (above 99th percentile) for better visualization
if 'VALOR' in df_viz.columns:
    valor_threshold = df_viz['VALOR'].quantile(0.99)
    df_viz = df_viz[df_viz['VALOR'] <= valor_threshold]

# 1. Portfolio Analysis by Unit - Total Value
plt.figure(figsize=(12, 8))
unit_values = df_viz.groupby('UNIDADE')['VALOR'].sum().sort_values(ascending=False)

# Bar plot for total values by unit
ax = sns.barplot(x=unit_values.index, y=unit_values.values, palette='viridis')
plt.title('Total Portfolio Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Total Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on top of bars
for i, v in enumerate(unit_values):
    ax.text(i, v + (unit_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=12)

plt.tight_layout()
plt.savefig(f'{viz_path}/total_portfolio_by_unit.png', dpi=300)
plt.close()

# 2. Average Contract Value by Unit
plt.figure(figsize=(12, 8))
avg_values = df_viz.groupby('UNIDADE')['VALOR'].mean().sort_values(ascending=False)

# Bar plot for average values by unit
ax = sns.barplot(x=avg_values.index, y=avg_values.values, palette='rocket')
plt.title('Average Contract Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Average Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on top of bars
for i, v in enumerate(avg_values):
    ax.text(i, v + (avg_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=12)

plt.tight_layout()
plt.savefig(f'{viz_path}/avg_contract_by_unit.png', dpi=300)
plt.close()

# 3. Contract Count by Operating Unit
plt.figure(figsize=(12, 8))
contract_counts = df.groupby('UNIDADE').size().sort_values(ascending=False)

# Bar plot for contract counts by unit
ax = sns.barplot(x=contract_counts.index, y=contract_counts.values, palette='mako')
plt.title('Number of Contracts by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Number of Contracts', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on top of bars
for i, v in enumerate(contract_counts):
    ax.text(i, v + (contract_counts.max() * 0.02), f'{v:,}', ha='center', fontsize=12)

plt.tight_layout()
plt.savefig(f'{viz_path}/contract_count_by_unit.png', dpi=300)
plt.close()

# 4. Time Series Analysis - Monthly Portfolio Value by Unit
# Filter data from the last 12 months for clearer visualization
if 'DATA_EMISSAO_NF' in df.columns:
    # Get last 12 months of data with valid dates and values
    df_recent = df[df['DATA_EMISSAO_NF'].notna() & df['VALOR'].notna()].copy()
    
    if not df_recent.empty:
        # Sort by date
        df_recent = df_recent.sort_values('DATA_EMISSAO_NF')
        
        # Group by month-year and unit
        monthly_values = df_recent.groupby(['MES_ANO', 'UNIDADE'])['VALOR'].sum().reset_index()
        
        # Create pivot table for easier plotting
        pivot_monthly = monthly_values.pivot(index='MES_ANO', columns='UNIDADE', values='VALOR')
        
        # Sort chronologically
        try:
            # Convert index to datetime for proper sorting
            pivot_monthly.index = pd.to_datetime(pivot_monthly.index, format='%m-%Y')
            pivot_monthly = pivot_monthly.sort_index()
            # Convert back to string format for display
            pivot_monthly.index = pivot_monthly.index.strftime('%m-%Y')
        except:
            pass
        
        # Plot time series
        plt.figure(figsize=(14, 10))
        pivot_monthly.plot(kind='line', marker='o', linewidth=2, figsize=(14, 10))
        plt.title('Monthly Portfolio Value by Operating Unit', fontsize=16)
        plt.xlabel('Month-Year', fontsize=14)
        plt.ylabel('Total Value (R$)', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Operating Unit', fontsize=12, title_fontsize=14)
        plt.xticks(rotation=45)
        
        # Format y-axis to show currency
        def currency_formatter(x, pos):
            return f'R$ {x:,.0f}'
        plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))
        
        plt.tight_layout()
        plt.savefig(f'{viz_path}/monthly_value_by_unit.png', dpi=300)
        plt.close()

# 5. Status Distribution by Unit
if 'STATUS' in df.columns:
    # Get top 5 statuses
    top_statuses = df['STATUS'].value_counts().nlargest(5).index.tolist()
    df_status = df[df['STATUS'].isin(top_statuses)].copy()
    
    plt.figure(figsize=(14, 10))
    status_counts = pd.crosstab(df_status['UNIDADE'], df_status['STATUS'])
    status_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='tab10')
    plt.title('Contract Status Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Status', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/status_distribution_by_unit.png', dpi=300)
    plt.close()

# 6. Operator Distribution by Unit
if 'OPERADORA' in df.columns:
    # Get top operators
    top_operators = df['OPERADORA'].value_counts().nlargest(5).index.tolist()
    df_operator = df[df['OPERADORA'].isin(top_operators)].copy()
    
    plt.figure(figsize=(14, 10))
    operator_counts = pd.crosstab(df_operator['UNIDADE'], df_operator['OPERADORA'])
    operator_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='Set3')
    plt.title('Insurance Operator Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Operator', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/operator_distribution_by_unit.png', dpi=300)
    plt.close()

# 7. Session Count Analysis
if 'NUM_SESSOES' in df.columns:
    plt.figure(figsize=(12, 8))
    session_avg = df.groupby('UNIDADE')['NUM_SESSOES'].mean().sort_values(ascending=False)
    
    ax = sns.barplot(x=session_avg.index, y=session_avg.values, palette='Blues_d')
    plt.title('Average Number of Sessions by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Average Number of Sessions', fontsize=14)
    plt.xticks(rotation=45)
    
    # Add value labels on top of bars
    for i, v in enumerate(session_avg):
        ax.text(i, v + 0.1, f'{v:.1f}', ha='center', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f'{viz_path}/avg_sessions_by_unit.png', dpi=300)
    plt.close()

# 8. Payment Status Analysis
if 'STATUS_FINAL_CONTA' in df.columns:
    plt.figure(figsize=(14, 10))
    payment_status = pd.crosstab(df['UNIDADE'], df['STATUS_FINAL_CONTA'])
    payment_status.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='RdYlGn')
    plt.title('Payment Status by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Payment Status', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/payment_status_by_unit.png', dpi=300)
    plt.close()

# 9. Create a comprehensive summary report with key metrics
summary_report = {}
for unit in df['UNIDADE'].unique():
    unit_data = df[df['UNIDADE'] == unit]
    
    summary_report[unit] = {
        'contracts': len(unit_data),
        'total_value': unit_data['VALOR'].sum(),
        'avg_value': unit_data['VALOR'].mean(),
        'avg_sessions': unit_data['NUM_SESSOES'].mean() if 'NUM_SESSOES' in unit_data.columns else None,
        'top_status': unit_data['STATUS'].value_counts().nlargest(1).index[0] if 'STATUS' in unit_data.columns else None,
        'top_operator': unit_data['OPERADORA'].value_counts().nlargest(1).index[0] if 'OPERADORA' in unit_data.columns else None
    }

# Convert summary to DataFrame for easier display and save to CSV
summary_df = pd.DataFrame.from_dict(summary_report, orient='index')
summary_df.to_csv(f'{viz_path}/portfolio_summary.csv')

# Display key performance indicators
print("=== KEY PORTFOLIO METRICS ===")
print(f"Total portfolio value: R$ {df['VALOR'].sum():,.2f}")
print(f"Total number of contracts: {len(df)}")
print(f"Average contract value: R$ {df['VALOR'].mean():,.2f}")
print("\nUnit Performance Summary:")
for unit, metrics in summary_report.items():
    print(f"\n{unit}:")
    print(f"  - Contracts: {metrics['contracts']:,}")
    print(f"  - Total value: R$ {metrics['total_value']:,.2f}")
    print(f"  - Average value: R$ {metrics['avg_value']:,.2f}")
    if metrics['avg_sessions']:
        print(f"  - Average sessions: {metrics['avg_sessions']:.1f}")
    if metrics['top_status']:
        print(f"  - Main status: {metrics['top_status']}")
    if metrics['top_operator']:
        print(f"  - Main operator: {metrics['top_operator']}")

print(f"\nVisualization files saved to: {viz_path}")
print(f"Processed data saved to: {processed_data_path}")

=== KEY PORTFOLIO METRICS ===
Total portfolio value: R$ 0.00
Total number of contracts: 5497
Average contract value: R$ nan

Unit Performance Summary:

ALPHAVILLE:
  - Contracts: 1,652
  - Total value: R$ 0.00
  - Average value: R$ nan
  - Average sessions: 14.9
  - Main status: ATIVO
  - Main operator: BRADESCO

BH:
  - Contracts: 807
  - Total value: R$ 0.00
  - Average value: R$ nan
  - Average sessions: 12.6
  - Main status: ATIVO
  - Main operator: BRADESCO REEMBOLSO

OSASCO:
  - Contracts: 5
  - Total value: R$ 0.00
  - Average value: R$ nan
  - Average sessions: 12.2
  - Main status: ATIVO
  - Main operator: AMIL

BRASILIA:
  - Contracts: 1
  - Total value: R$ 0.00
  - Average value: R$ nan
  - Average sessions: 13.0
  - Main status: ATIVO
  - Main operator: AMIL

DOMICILIAR:
  - Contracts: 1,622
  - Total value: R$ 0.00
  - Average value: R$ nan
  - Average sessions: 16.3
  - Main status: ATIVO
  - Main operator: BRADESCO

SAO JOSE DOS CAMPOS:
  - Contracts: 841
  - Total value

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [6]:
# Fix the VALOR conversion issue and create a comprehensive portfolio analysis report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import re

# Load the original Excel file again
file_path = '/data/chats/igk4wd/workspace/uploads/Teste Power BI - Deveras RH.xlsx'
location_sheets = ['Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']

# Create standardized dataframes
standardized_dfs = {}

# Define an improved function to convert VALOR to numeric
def convert_valor_to_numeric(valor):
    if pd.isna(valor):
        return np.nan
    if isinstance(valor, (int, float)):
        return float(valor)
    
    # Convert string to string format
    valor_str = str(valor)
    
    # Remove any non-numeric characters except . and ,
    clean_valor = re.sub(r'[^\d.,]', '', valor_str)
    
    # Handle Brazilian/European format (e.g. 1.234,56)
    if ',' in clean_valor and '.' in clean_valor:
        # If there are both dots and commas, assume Brazilian/European format
        clean_valor = clean_valor.replace('.', '').replace(',', '.')
    elif ',' in clean_valor:
        # If only commas, replace with dots
        clean_valor = clean_valor.replace(',', '.')
    
    # Convert to float
    try:
        return float(clean_valor)
    except:
        return np.nan

# Process each sheet and standardize them
for sheet in location_sheets:
    print(f"Processing {sheet} sheet...")
    
    # Read the sheet
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    # Process VALOR column
    if 'VALOR' in df.columns:
        # Save original values for reference
        df['VALOR_ORIGINAL'] = df['VALOR']
        
        # Convert to numeric
        df['VALOR'] = df['VALOR'].apply(convert_valor_to_numeric)
        
        # Check conversion results
        print(f"  - Sample original values: {df['VALOR_ORIGINAL'].head(3).tolist()}")
        print(f"  - Sample converted values: {df['VALOR'].head(3).tolist()}")
        print(f"  - NaN values after conversion: {df['VALOR'].isna().sum()}")
        print(f"  - VALOR statistics: Min={df['VALOR'].min()}, Max={df['VALOR'].max()}, Mean={df['VALOR'].mean():.2f}")
    
    # Standardize UNIDADE column
    if 'UNIDADE' in df.columns:
        df['UNIDADE'] = df['UNIDADE'].str.strip().str.upper()
        # Map variations to standard names
        unit_mapping = {
            'ALPHA': 'ALPHAVILLE',
            'alpha ': 'ALPHAVILLE',
            'SJC': 'SAO JOSE DOS CAMPOS',
            'BELO HORIZONTE': 'BH',
            'BSB': 'BRASILIA'
        }
        df['UNIDADE'] = df['UNIDADE'].replace(unit_mapping)
    
    # Store the standardized dataframe
    standardized_dfs[sheet] = df

# Combine all sheets into a single dataframe
combined_df = pd.concat(standardized_dfs.values(), ignore_index=True)

# Create a directory for visualizations
import os
viz_path = '/data/chats/igk4wd/workspace/visualizations'
os.makedirs(viz_path, exist_ok=True)

# Set visualization style
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 12})

# Filter extreme values for better visualization (excluding top 1%)
df_viz = combined_df.copy()
if 'VALOR' in df_viz.columns:
    valor_threshold = df_viz['VALOR'].quantile(0.99)
    df_viz = df_viz[df_viz['VALOR'] <= valor_threshold]

# 1. Portfolio Value by Operating Unit
plt.figure(figsize=(12, 8))
unit_values = df_viz.groupby('UNIDADE')['VALOR'].sum().sort_values(ascending=False)

ax = sns.barplot(x=unit_values.index, y=unit_values.values, palette='viridis')
plt.title('Total Portfolio Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Total Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(unit_values):
    ax.text(i, v + (unit_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/total_portfolio_by_unit.png', dpi=300)
plt.close()

# 2. Average Contract Value by Unit
plt.figure(figsize=(12, 8))
avg_values = df_viz.groupby('UNIDADE')['VALOR'].mean().sort_values(ascending=False)

ax = sns.barplot(x=avg_values.index, y=avg_values.values, palette='rocket')
plt.title('Average Contract Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Average Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(avg_values):
    ax.text(i, v + (avg_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/avg_contract_by_unit.png', dpi=300)
plt.close()

# 3. Number of Contracts by Unit
plt.figure(figsize=(12, 8))
contract_counts = combined_df.groupby('UNIDADE').size().sort_values(ascending=False)

ax = sns.barplot(x=contract_counts.index, y=contract_counts.values, palette='mako')
plt.title('Number of Contracts by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Number of Contracts', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(contract_counts):
    ax.text(i, v + (contract_counts.max() * 0.02), f'{v:,}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/contract_count_by_unit.png', dpi=300)
plt.close()

# 4. Status Distribution by Unit
if 'STATUS' in combined_df.columns:
    # Get top 5 statuses
    top_statuses = combined_df['STATUS'].value_counts().nlargest(5).index.tolist()
    df_status = combined_df[combined_df['STATUS'].isin(top_statuses)].copy()
    
    plt.figure(figsize=(14, 10))
    status_counts = pd.crosstab(df_status['UNIDADE'], df_status['STATUS'])
    status_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='tab10')
    plt.title('Contract Status Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Status', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/status_distribution_by_unit.png', dpi=300)
    plt.close()

# 5. Operator Distribution by Unit
if 'OPERADORA' in combined_df.columns:
    # Get top operators
    top_operators = combined_df['OPERADORA'].value_counts().nlargest(5).index.tolist()
    df_operator = combined_df[combined_df['OPERADORA'].isin(top_operators)].copy()
    
    plt.figure(figsize=(14, 10))
    operator_counts = pd.crosstab(df_operator['UNIDADE'], df_operator['OPERADORA'])
    operator_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='Set3')
    plt.title('Insurance Operator Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Operator', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/operator_distribution_by_unit.png', dpi=300)
    plt.close()

# 6. Sessions Distribution by Unit
if 'Nº SESSÕES' in combined_df.columns:
    plt.figure(figsize=(12, 8))
    session_avg = combined_df.groupby('UNIDADE')['Nº SESSÕES'].mean().sort_values(ascending=False)
    
    ax = sns.barplot(x=session_avg.index, y=session_avg.values, palette='Blues_d')
    plt.title('Average Number of Sessions by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Average Number of Sessions', fontsize=14)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(session_avg):
        ax.text(i, v + 0.1, f'{v:.1f}', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.savefig(f'{viz_path}/avg_sessions_by_unit.png', dpi=300)
    plt.close()

# 7. Boxplot of VALOR by Unit
plt.figure(figsize=(14, 10))
sns.boxplot(x='UNIDADE', y='VALOR', data=df_viz, palette='Set2')
plt.title('Distribution of Contract Values by Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Contract Value (R$)', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{viz_path}/valor_distribution_by_unit.png', dpi=300)
plt.close()

# Create the comprehensive summary report
summary_data = {}

for unit in combined_df['UNIDADE'].unique():
    unit_data = combined_df[combined_df['UNIDADE'] == unit]
    
    # Calculate key metrics
    summary_data[unit] = {
        'n_contracts': len(unit_data),
        'total_value': unit_data['VALOR'].sum(),
        'avg_value': unit_data['VALOR'].mean(),
        'max_value': unit_data['VALOR'].max(),
        'min_value': unit_data['VALOR'].min()
    }
    
    # Add sessions data if available
    if 'Nº SESSÕES' in unit_data.columns:
        summary_data[unit]['avg_sessions'] = unit_data['Nº SESSÕES'].mean()
    
    # Add status distribution if available
    if 'STATUS' in unit_data.columns and not unit_data['STATUS'].isna().all():
        top_status = unit_data['STATUS'].value_counts().index[0] if not unit_data['STATUS'].value_counts().empty else 'N/A'
        summary_data[unit]['top_status'] = top_status
    
    # Add operator distribution if available
    if 'OPERADORA' in unit_data.columns and not unit_data['OPERADORA'].isna().all():
        top_operator = unit_data['OPERADORA'].value_counts().index[0] if not unit_data['OPERADORA'].value_counts().empty else 'N/A'
        summary_data[unit]['top_operator'] = top_operator

# Convert to DataFrame and save
summary_df = pd.DataFrame.from_dict(summary_data, orient='index')
summary_df.to_csv(f'{viz_path}/portfolio_summary.csv')

# Display key portfolio metrics
print("\n=== PORTFOLIO PERFORMANCE ANALYSIS ===")
print(f"Total portfolio value: R$ {combined_df['VALOR'].sum():,.2f}")
print(f"Total number of contracts: {len(combined_df):,}")
print(f"Average contract value: R$ {combined_df['VALOR'].mean():,.2f}")

# Display unit performance summary
print("\n=== UNIT PERFORMANCE SUMMARY ===")
for unit, metrics in summary_data.items():
    print(f"\n{unit}:")
    print(f"  - Contracts: {metrics['n_contracts']:,}")
    print(f"  - Total value: R$ {metrics['total_value']:,.2f}")
    print(f"  - Average value: R$ {metrics['avg_value']:,.2f}")
    
    if 'avg_sessions' in metrics:
        print(f"  - Average sessions: {metrics['avg_sessions']:.1f}")
    
    if 'top_status' in metrics:
        print(f"  - Main status: {metrics['top_status']}")
    
    if 'top_operator' in metrics:
        print(f"  - Main operator: {metrics['top_operator']}")

# Save the processed data
processed_data_path = '/data/chats/igk4wd/workspace/processed_data.csv'
combined_df.to_csv(processed_data_path, index=False)
print(f"\nProcessed data saved to: {processed_data_path}")
print(f"Visualization files saved to: {viz_path}")

# List the identified data inconsistencies
print("\n=== DATA INCONSISTENCIES ===")
print("1. Inconsistent column naming across sheets")
print("2. Inconsistent UNIDADE names (e.g., 'ALPHAVILLE', 'alpha ')")
print("3. Multiple patient names assigned to the same client ID")
print("4. Inconsistent data types for common columns across sheets")
print("5. Unusual date values (dates before 2000 likely errors)")
print("6. Extreme values in VALOR column that required filtering")
print("7. Missing values in critical columns")

Processing Alphaville sheet...
  - Sample original values: [41545.78, 54270.99, 43074.45]
  - Sample converted values: [41545.78, 54270.99, 43074.45]
  - NaN values after conversion: 21
  - VALOR statistics: Min=3000.0, Max=124526.12, Mean=47585.68
Processing BH sheet...
  - Sample original values: [51231.24, 47328.33, 57888.45]
  - Sample converted values: [51231.24, 47328.33, 57888.45]
  - NaN values after conversion: 52
  - VALOR statistics: Min=8.0, Max=84354.34, Mean=28020.54
Processing SJC sheet...
  - Sample original values: [datetime.datetime(2014, 1, 20, 9, 21, 36), datetime.datetime(2016, 5, 29, 18, 57, 36), datetime.datetime(2016, 8, 29, 9, 21, 36)]
  - Sample converted values: [20140120092136.0, 20160529185736.0, 20160829092136.0]
  - NaN values after conversion: 18
  - VALOR statistics: Min=2600.0, Max=26030602074048.0, Mean=11384180980684.39
Processing PAULINIA sheet...
  - Sample original values: [78661.77, 55947.52, 54432.92]
  - Sample converted values: [78661.77, 5594

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [7]:
# Fix the extreme VALOR values in SJC and PAULINIA sheets 
# and create a final comprehensive portfolio analysis report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import re
import os
from datetime import datetime

# Load the original Excel file again to fix the data issues
file_path = '/data/chats/igk4wd/workspace/uploads/Teste Power BI - Deveras RH.xlsx'
location_sheets = ['Alphaville', 'BH', 'SJC', 'PAULINIA', 'CAMPINAS']

# Create standardized dataframes
standardized_dfs = {}

# Improved function to convert VALOR to numeric, handling special cases
def convert_valor_to_numeric(valor):
    if pd.isna(valor):
        return np.nan
    
    # If it's already a number, but actually a date
    if isinstance(valor, (int, float)) and valor > 1000000:  # Very large numbers are likely date formats
        return np.nan
    
    # If it's already a reasonable number
    if isinstance(valor, (int, float)) and valor < 1000000:
        return float(valor)
    
    # If it's a datetime object (common issue in some sheets)
    if isinstance(valor, datetime):
        return np.nan
    
    # Convert string to string format for processing
    valor_str = str(valor)
    
    # Check if it might be a date in disguise
    if len(valor_str) > 10 and re.match(r'\d{10,}', valor_str):
        return np.nan
    
    # Remove any non-numeric characters except . and ,
    clean_valor = re.sub(r'[^\d.,]', '', valor_str)
    
    # Handle Brazilian/European format (e.g. 1.234,56)
    if ',' in clean_valor and '.' in clean_valor:
        clean_valor = clean_valor.replace('.', '').replace(',', '.')
    elif ',' in clean_valor:
        clean_valor = clean_valor.replace(',', '.')
    
    # Convert to float
    try:
        val = float(clean_valor)
        # Additional sanity check - extreme values are likely errors
        if val > 500000:  # Assuming no contract should be above 500,000
            return np.nan
        return val
    except:
        return np.nan

# Process each sheet and standardize them
print("=== DATA CLEANING AND STANDARDIZATION ===")
for sheet in location_sheets:
    print(f"\nProcessing {sheet} sheet...")
    
    # Read the sheet
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    # Process VALOR column
    if 'VALOR' in df.columns:
        # Save original values for reference
        df['VALOR_ORIGINAL'] = df['VALOR']
        
        # Convert to numeric with improved function
        df['VALOR'] = df['VALOR'].apply(convert_valor_to_numeric)
        
        # Check conversion results
        print(f"  - Original values (sample): {df['VALOR_ORIGINAL'].head(3).tolist()}")
        print(f"  - Converted values (sample): {df['VALOR'].head(3).tolist()}")
        print(f"  - NaN values after conversion: {df['VALOR'].isna().sum()} out of {len(df)}")
        print(f"  - VALOR statistics: Min={df['VALOR'].min():.2f}, Max={df['VALOR'].max():.2f}, Mean={df['VALOR'].mean():.2f}")
    
    # Clean and standardize column names
    df.columns = [col.strip() for col in df.columns]
    
    # Standardize UNIDADE column
    if 'UNIDADE' in df.columns:
        df['UNIDADE'] = df['UNIDADE'].str.strip().str.upper()
        # Map variations to standard names
        unit_mapping = {
            'ALPHA': 'ALPHAVILLE',
            'alpha ': 'ALPHAVILLE',
            'SJC': 'SAO JOSE DOS CAMPOS',
            'BELO HORIZONTE': 'BH',
            'BSB': 'BRASILIA'
        }
        df['UNIDADE'] = df['UNIDADE'].replace(unit_mapping)
    
    # Add source sheet information
    df['FONTE_PLANILHA'] = sheet
    
    # Store the cleaned dataframe
    standardized_dfs[sheet] = df

# Combine all sheets into a single dataframe
combined_df = pd.concat(standardized_dfs.values(), ignore_index=True)

# Further cleaning: Remove rows with NaN VALOR and filter extreme values
valid_valor_df = combined_df[combined_df['VALOR'].notna()].copy()
print(f"\nRows with valid VALOR: {len(valid_valor_df)} out of {len(combined_df)} total rows")

# Create directory for visualizations and reports
viz_path = '/data/chats/igk4wd/workspace/visualizations'
os.makedirs(viz_path, exist_ok=True)

# Set visualization style
sns.set(style="whitegrid")
plt.rcParams.update({'font.size': 12})

# ====== PORTFOLIO ANALYSIS VISUALIZATIONS ======

# 1. Portfolio Value by Operating Unit
plt.figure(figsize=(12, 8))
# Group by UNIDADE and sum the VALOR, handling NaN values
unit_values = valid_valor_df.groupby('UNIDADE')['VALOR'].sum().sort_values(ascending=False)

ax = sns.barplot(x=unit_values.index, y=unit_values.values, palette='viridis')
plt.title('Total Portfolio Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Total Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(unit_values):
    ax.text(i, v + (unit_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/total_portfolio_by_unit.png', dpi=300)
plt.close()

# 2. Average Contract Value by Unit
plt.figure(figsize=(12, 8))
avg_values = valid_valor_df.groupby('UNIDADE')['VALOR'].mean().sort_values(ascending=False)

ax = sns.barplot(x=avg_values.index, y=avg_values.values, palette='rocket')
plt.title('Average Contract Value by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Average Value (R$)', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(avg_values):
    ax.text(i, v + (avg_values.max() * 0.02), f'R$ {v:,.0f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/avg_contract_by_unit.png', dpi=300)
plt.close()

# 3. Number of Contracts by Unit
plt.figure(figsize=(12, 8))
contract_counts = combined_df.groupby('UNIDADE').size().sort_values(ascending=False)

ax = sns.barplot(x=contract_counts.index, y=contract_counts.values, palette='mako')
plt.title('Number of Contracts by Operating Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Number of Contracts', fontsize=14)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(contract_counts):
    ax.text(i, v + (contract_counts.max() * 0.02), f'{v:,}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig(f'{viz_path}/contract_count_by_unit.png', dpi=300)
plt.close()

# 4. Status Distribution by Unit (Top 5)
if 'STATUS' in combined_df.columns:
    # Get top 5 statuses
    top_statuses = combined_df['STATUS'].value_counts().nlargest(5).index.tolist()
    df_status = combined_df[combined_df['STATUS'].isin(top_statuses)].copy()
    
    plt.figure(figsize=(14, 10))
    status_counts = pd.crosstab(df_status['UNIDADE'], df_status['STATUS'])
    status_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='tab10')
    plt.title('Contract Status Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Status', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/status_distribution_by_unit.png', dpi=300)
    plt.close()

# 5. Operator Distribution by Unit (Top 5)
if 'OPERADORA' in combined_df.columns:
    # Get top operators
    top_operators = combined_df['OPERADORA'].value_counts().nlargest(5).index.tolist()
    df_operator = combined_df[combined_df['OPERADORA'].isin(top_operators)].copy()
    
    plt.figure(figsize=(14, 10))
    operator_counts = pd.crosstab(df_operator['UNIDADE'], df_operator['OPERADORA'])
    operator_counts.plot(kind='bar', stacked=True, figsize=(14, 10), colormap='Set3')
    plt.title('Insurance Operator Distribution by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Number of Contracts', fontsize=14)
    plt.legend(title='Operator', fontsize=12, title_fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{viz_path}/operator_distribution_by_unit.png', dpi=300)
    plt.close()

# 6. Sessions Distribution by Unit
if 'Nº SESSÕES' in combined_df.columns:
    plt.figure(figsize=(12, 8))
    session_avg = combined_df.groupby('UNIDADE')['Nº SESSÕES'].mean().sort_values(ascending=False)
    
    ax = sns.barplot(x=session_avg.index, y=session_avg.values, palette='Blues_d')
    plt.title('Average Number of Sessions by Operating Unit', fontsize=16)
    plt.xlabel('Operating Unit', fontsize=14)
    plt.ylabel('Average Number of Sessions', fontsize=14)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(session_avg):
        ax.text(i, v + 0.1, f'{v:.1f}', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.savefig(f'{viz_path}/avg_sessions_by_unit.png', dpi=300)
    plt.close()

# 7. Boxplot of VALOR by Unit
plt.figure(figsize=(14, 10))
sns.boxplot(x='UNIDADE', y='VALOR', data=valid_valor_df, palette='Set2')
plt.title('Distribution of Contract Values by Unit', fontsize=16)
plt.xlabel('Operating Unit', fontsize=14)
plt.ylabel('Contract Value (R$)', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{viz_path}/valor_distribution_by_unit.png', dpi=300)
plt.close()

# Create a comprehensive summary report for each operating unit
summary_data = {}

for unit in combined_df['UNIDADE'].unique():
    unit_data = combined_df[combined_df['UNIDADE'] == unit]
    unit_valid_valor = unit_data[unit_data['VALOR'].notna()]
    
    # Calculate key metrics
    summary_data[unit] = {
        'n_contracts': len(unit_data),
        'n_valid_contracts': len(unit_valid_valor),
        'total_value': unit_valid_valor['VALOR'].sum(),
        'avg_value': unit_valid_valor['VALOR'].mean(),
        'max_value': unit_valid_valor['VALOR'].max(),
        'min_value': unit_valid_valor['VALOR'].min(),
        'data_completeness': len(unit_valid_valor) / len(unit_data) if len(unit_data) > 0 else 0,
    }
    
    # Add sessions data if available
    if 'Nº SESSÕES' in unit_data.columns:
        summary_data[unit]['avg_sessions'] = unit_data['Nº SESSÕES'].mean()
    
    # Add status distribution if available
    if 'STATUS' in unit_data.columns and not unit_data['STATUS'].isna().all():
        status_counts = unit_data['STATUS'].value_counts()
        top_status = status_counts.index[0] if not status_counts.empty else 'N/A'
        top_status_pct = status_counts.iloc[0] / len(unit_data) if not status_counts.empty else 0
        summary_data[unit]['top_status'] = top_status
        summary_data[unit]['top_status_pct'] = top_status_pct
    
    # Add operator distribution if available
    if 'OPERADORA' in unit_data.columns and not unit_data['OPERADORA'].isna().all():
        operator_counts = unit_data['OPERADORA'].value_counts()
        top_operator = operator_counts.index[0] if not operator_counts.empty else 'N/A'
        top_operator_pct = operator_counts.iloc[0] / len(unit_data) if not operator_counts.empty else 0
        summary_data[unit]['top_operator'] = top_operator
        summary_data[unit]['top_operator_pct'] = top_operator_pct

# Convert to DataFrame for reporting
summary_df = pd.DataFrame.from_dict(summary_data, orient='index')
summary_df.to_csv(f'{viz_path}/portfolio_summary.csv')

# Create a comprehensive final report
report_path = f'{viz_path}/portfolio_analysis_report.md'
with open(report_path, 'w') as f:
    f.write("# Portfolio Analysis Report - Deveras RH\n\n")
    
    f.write("## Data Overview\n")
    f.write(f"- Total contracts analyzed: {len(combined_df):,}\n")
    f.write(f"- Contracts with valid financial data: {len(valid_valor_df):,} ({len(valid_valor_df)/len(combined_df):.1%})\n")
    f.write(f"- Operating units: {', '.join(combined_df['UNIDADE'].unique())}\n")
    f.write(f"- Total portfolio value: R$ {valid_valor_df['VALOR'].sum():,.2f}\n\n")
    
    f.write("## Key Portfolio Metrics by Operating Unit\n\n")
    
    for unit, metrics in summary_data.items():
        f.write(f"### {unit}\n")
        f.write(f"- Contracts: {metrics['n_contracts']:,}\n")
        f.write(f"- Contracts with valid financial data: {metrics['n_valid_contracts']:,} ({metrics['data_completeness']:.1%})\n")
        f.write(f"- Total value: R$ {metrics['total_value']:,.2f}\n")
        f.write(f"- Average value per contract: R$ {metrics['avg_value']:,.2f}\n")
        f.write(f"- Value range: R$ {metrics['min_value']:,.2f} to R$ {metrics['max_value']:,.2f}\n")
        
        if 'avg_sessions' in metrics:
            f.write(f"- Average sessions: {metrics['avg_sessions']:.1f}\n")
        
        if 'top_status' in metrics:
            f.write(f"- Primary status: {metrics['top_status']} ({metrics['top_status_pct']:.1%} of contracts)\n")
        
        if 'top_operator' in metrics:
            f.write(f"- Primary operator: {metrics['top_operator']} ({metrics['top_operator_pct']:.1%} of contracts)\n")
        
        f.write("\n")
    
    f.write("## Data Inconsistencies Identified\n\n")
    f.write("1. **Inconsistent Column Naming**: Different sheets used varied naming conventions for the same data.\n")
    f.write("2. **Inconsistent Location Names**: The same operating unit appeared under different names (e.g., 'ALPHAVILLE', 'alpha').\n")
    f.write("3. **Client ID Inconsistencies**: Multiple patient names were assigned to the same client ID.\n")
    f.write("4. **Data Type Inconsistencies**: The same columns had different data types across sheets.\n")
    f.write("5. **Problematic Date Values**: Some sheets contained unusual date values (dates before 2000).\n")
    f.write("6. **Extreme Values in VALOR Column**: SJC and PAULINIA sheets contained datetime values incorrectly interpreted as currency.\n")
    f.write("7. **Missing Values**: Critical columns contained significant numbers of missing values.\n\n")
    
    f.write("## Recommendations for Dashboard Creation\n\n")
    f.write("1. Use the standardized processed data provided in the CSV file for consistent analysis.\n")
    f.write("2. Implement filters by operating unit, status, and insurance operator.\n")
    f.write("3. Include time series visualizations to track portfolio growth over time.\n")
    f.write("4. Create separate panels for individual operating unit performance and consolidated views.\n")
    f.write("5. Consider implementing alerts for contracts with unusual values or statuses.\n")
    f.write("6. Include data quality metrics to monitor completeness and consistency.\n")

# Save the processed data with fixed values
processed_data_path = '/data/chats/igk4wd/workspace/processed_data.csv'
combined_df.to_csv(processed_data_path, index=False)

# Display summary of the analysis
print("\n=== PORTFOLIO PERFORMANCE ANALYSIS ===")
print(f"Total portfolio value: R$ {valid_valor_df['VALOR'].sum():,.2f}")
print(f"Total number of contracts: {len(combined_df):,}")
print(f"Contracts with valid financial data: {len(valid_valor_df):,} ({len(valid_valor_df)/len(combined_df):.1%})")
print(f"Average contract value: R$ {valid_valor_df['VALOR'].mean():,.2f}")

# Display unit performance summary
print("\n=== UNIT PERFORMANCE SUMMARY ===")
for unit, metrics in summary_data.items():
    print(f"\n{unit}:")
    print(f"  - Contracts: {metrics['n_contracts']:,}")
    print(f"  - Valid financial data: {metrics['n_valid_contracts']:,} ({metrics['data_completeness']:.1%})")
    print(f"  - Total value: R$ {metrics['total_value']:,.2f}")
    print(f"  - Average value: R$ {metrics['avg_value']:,.2f}")
    
    if 'avg_sessions' in metrics:
        print(f"  - Average sessions: {metrics['avg_sessions']:.1f}")
    
    if 'top_status' in metrics:
        print(f"  - Main status: {metrics['top_status']}")
    
    if 'top_operator' in metrics:
        print(f"  - Main operator: {metrics['top_operator']}")

print(f"\nDetailed report saved to: {report_path}")
print(f"Processed data saved to: {processed_data_path}")
print(f"Visualization files saved to: {viz_path}")

=== DATA CLEANING AND STANDARDIZATION ===

Processing Alphaville sheet...
  - Original values (sample): [41545.78, 54270.99, 43074.45]
  - Converted values (sample): [41545.78, 54270.99, 43074.45]
  - NaN values after conversion: 21 out of 1651
  - VALOR statistics: Min=3000.00, Max=124526.12, Mean=47585.68

Processing BH sheet...
  - Original values (sample): [51231.24, 47328.33, 57888.45]
  - Converted values (sample): [51231.24, 47328.33, 57888.45]
  - NaN values after conversion: 52 out of 814
  - VALOR statistics: Min=8.00, Max=84354.34, Mean=28020.54

Processing SJC sheet...
  - Original values (sample): [datetime.datetime(2014, 1, 20, 9, 21, 36), datetime.datetime(2016, 5, 29, 18, 57, 36), datetime.datetime(2016, 8, 29, 9, 21, 36)]
  - Converted values (sample): [nan, nan, nan]
  - NaN values after conversion: 1458 out of 2587
  - VALOR statistics: Min=2600.00, Max=158214.06, Mean=44010.50

Processing PAULINIA sheet...
  - Original values (sample): [78661.77, 55947.52, 54432.92]

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>